In [ ]:
import polars as pl

In [ ]:
df = pl.read_parquet("kjoretoyinfo_fra_2020.parquet")
df

In [ ]:
# Denne er viktig fordi data er ikke unikt identifisert av kolonnene våre
# Vi får trøbbel når vi skal gjøre group by senere - noen (forskjellige) biler har identiske data. 
df = df.with_row_index("id")
df

In [ ]:
# Vi leser inn fargekodene
fargekode = pl.read_csv("fargekode.csv", separator=";", dtypes={"kode":str})
fargekode

In [ ]:
# Legg merke til at tekn_farge i df har 0 først. 
# Dette må vi fikse også i fargekode for å kunne joine.
# Vi vil også bytte navn på "beskrivelse"-kolonna til noe mer fornuftig. 
fargekode = fargekode.with_columns(
    pl.col("kode").str.pad_start(2, "0")
).rename(
    {"beskrivelse":"farge_navn"}
)
fargekode

In [ ]:
# Her ser vi at vi har litt trøbbel
# Noen kjøretøy har flere farger! 
df.sort(pl.col("tekn_farge").str.len_chars(), nulls_last=True, descending=True)

In [ ]:
# Vi må gjøre en liten jobb for å kunne joine hver fargekode med fargen sin. 
# Først så lager vi en nøstet liste inne i hver kolonne ved å splitte på.
df = df.with_columns(pl.col("tekn_farge").str.split(","))
df

In [ ]:
# Her er et eksempel på hvordan slike rader ser ut nå:
df.sort(pl.col("tekn_farge").list.len(), nulls_last=True, descending=True).head(2).select("id", "tekn_farge")

In [ ]:
# Nå "sprenger" vi denne liste-kolonnen, og lager i stedet for en ny rad pr. listeelement i tekn_farge hvor de andre radene er identiske. 
df = df.explode("tekn_farge")
df

In [ ]:
# Sammenlign en av radene som hadde mange farger tidligere
df.filter(pl.col("id") == pl.lit(95018)).select("id", "tekn_farge")

In [ ]:
# Med left join så mister vi ingen biler vi ikke vet fargen på. 
df = df.join(fargekode, left_on="tekn_farge", right_on="kode", how="left")
df

In [ ]:
# Vi observerer at vi har fått tak i farge_navn for en av bilene med flere farger. 
# Men nå vil vi normalisere litt igjen. 
df.filter(pl.col("id") == pl.lit(95018)).select("id", "tekn_farge", "farge_navn")

In [ ]:
# Vi grupperer på alle kolonner som ikke er tekn farge og farge navn.
# Se https://docs.pola.rs/user-guide/expressions/column-selections/
# Vi aggregerer farge_navn og tekn_farge - dette samler bare opp kolonnene som nøstede kolonner (liste-kolonner). 
# Fra liste-kolonnene kan vi joine elementene inne i listene til strenger. 
df = df.group_by(pl.col("*").exclude("tekn_farge", "farge_navn")).agg(
    pl.col("farge_navn"),
    pl.col("tekn_farge"),
).with_columns(
    pl.col("farge_navn").list.join(","),
    pl.col("tekn_farge").list.join(",")
)
df

In [ ]:
# Vi sjekker hvordan bilen vår endte opp:
df.filter(pl.col("id") == pl.lit(95018)).select("id", "tekn_farge", "farge_navn")